In [1]:
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, RocCurveDisplay

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


In [4]:
df = pd.read_csv("train.csv")


In [5]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
df.info()
df.describe(include="all")
df['Survived'].value_counts(normalize=True)

print(df.groupby('Sex')['Survived'].mean().sort_values(ascending=False))

print(df.groupby('Pclass')['Survived'].mean().sort_values(ascending=False))

df.isna().mean().sort_values(ascending=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64
Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64


Cabin          0.771044
Age            0.198653
Embarked       0.002245
PassengerId    0.000000
Name           0.000000
Pclass         0.000000
Survived       0.000000
Sex            0.000000
Parch          0.000000
SibSp          0.000000
Fare           0.000000
Ticket         0.000000
dtype: float64

In [8]:
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

def extract_title(name: str) -> str:
    title = name.split(',')[1].split('.')[0].strip()
    rare = ['Lady','Countess','Capt','Col','Don','Dr','Major','Rev','Sir','Jonkheer','Dona']
    if title in rare:
        return 'Rare'
    if title in ['Mlle', 'Ms']:
        return 'Miss'
    if title == 'Mme':
        return 'Mrs'
    return title

df['Title'] = df['Name'].apply(extract_title)


In [9]:
X = df[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','FamilySize','Title']]
y = df['Survived'].astype(int)

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [10]:
numeric_features = ['Age','SibSp','Parch','Fare','FamilySize','Pclass']
categorical_features = ['Sex','Embarked','Title']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore'))
])

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)


In [11]:
log_reg = Pipeline(steps=[
    ('prep', preprocess),
    ('model', LogisticRegression(max_iter=200, solver='liblinear'))
])

log_reg.fit(X_train, y_train)
pred_lr = log_reg.predict(X_valid)
proba_lr = log_reg.predict_proba(X_valid)[:,1]

print("Logistic Regression — Accuracy:", accuracy_score(y_valid, pred_lr))
print("Logistic Regression — ROC-AUC:", roc_auc_score(y_valid, proba_lr))
print("\nConfusion Matrix:\n", confusion_matrix(y_valid, pred_lr))
print("\nClassification Report:\n", classification_report(y_valid, pred_lr))




Logistic Regression — Accuracy: 0.8435754189944135
Logistic Regression — ROC-AUC: 0.8811594202898552

Confusion Matrix:
 [[100  10]
 [ 18  51]]

Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.91      0.88       110
           1       0.84      0.74      0.78        69

    accuracy                           0.84       179
   macro avg       0.84      0.82      0.83       179
weighted avg       0.84      0.84      0.84       179



In [12]:
tree_pipe = Pipeline(steps=[
    ('prep', preprocess),
    ('model', DecisionTreeClassifier(random_state=42))
])

param_grid = {
    'model__max_depth': [3, 4, 5, 6, None],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 5]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(
    estimator=tree_pipe,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=cv,
    n_jobs=-1
)

grid.fit(X_train, y_train)

best_tree = grid.best_estimator_
pred_dt = best_tree.predict(X_valid)
proba_dt = best_tree.predict_proba(X_valid)[:,1]

print("Decision Tree — Best Params:", grid.best_params_)
print("Decision Tree — Accuracy:", accuracy_score(y_valid, pred_dt))
print("Decision Tree — ROC-AUC:", roc_auc_score(y_valid, proba_dt))
print("\nConfusion Matrix:\n", confusion_matrix(y_valid, pred_dt))
print("\nClassification Report:\n", classification_report(y_valid, pred_dt))


Decision Tree — Best Params: {'model__max_depth': 3, 'model__min_samples_leaf': 5, 'model__min_samples_split': 2}
Decision Tree — Accuracy: 0.8379888268156425
Decision Tree — ROC-AUC: 0.8657444005270092

Confusion Matrix:
 [[97 13]
 [16 53]]

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.88      0.87       110
           1       0.80      0.77      0.79        69

    accuracy                           0.84       179
   macro avg       0.83      0.82      0.83       179
weighted avg       0.84      0.84      0.84       179



In [13]:
scores_lr = cross_val_score(log_reg, X, y, cv=cv, scoring='roc_auc', n_jobs=-1)
scores_dt = cross_val_score(best_tree, X, y, cv=cv, scoring='roc_auc', n_jobs=-1)

print(f"LogReg CV ROC-AUC: mean={scores_lr.mean():.3f} ± {scores_lr.std():.3f}")
print(f"DecTree CV ROC-AUC: mean={scores_dt.mean():.3f} ± {scores_dt.std():.3f}")


LogReg CV ROC-AUC: mean=0.873 ± 0.019
DecTree CV ROC-AUC: mean=0.853 ± 0.018
